In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

# Creating the investment analysis crew

## Planner and Replanner

In [3]:
from langchain_openai import ChatOpenAI
planner_llm = ChatOpenAI(model="gpt-5-nano", temperature=0)

In [11]:
from typing import TypedDict, List, Literal, Union
from pydantic import BaseModel, Field

class Respond(BaseModel):
    response: str
    
class UseAgent(BaseModel):
    agent_to_use: Literal[ "technical_analysis_agent", "fundamental_analysis_agent"] = Field(..., description="Agent to use")
    query_to_send: str = Field(..., description="The query to ask this agent")

class Consolidate(BaseModel):
    query_to_send: str = Field(..., description="Consolidation instructions")

class Step(BaseModel):
    step_number: int = Field(..., description="The step number in the plan")
    action: Union[UseAgent, Respond, Consolidate] = Field(
        ..., 
        description= (
            "Action to perform. Use `Respond` if you want to respond to the user. "
            "If you need tools, use `UseAgent`. Always end with a `Consolidate` if "
            "you are invoking multiple agents, or a single agent more than once."
        )
    )

class Plan(BaseModel):
    steps: List[Step] = Field(..., description="The list of steps in the plan")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

planner_prompt = ChatPromptTemplate.from_template(
    """For a given objective, come up with a simple step by step plan to achieve the objective.
    
    The plan should involve individual tasks that if executed correctly will yield the final outcome.
    
    <important>
        Invoke the fundamental_analysis_agent and technical_analysis_agent only once per company of interest.
        If there are two companies of interest, invoke the respective agent/s twice.
        Always use both agents to analyze stock market data when evaluating investments.
        Always add a consolidation step if you are invoking more than one agent or you're invoking an agent more than once.
    </important>
    
    This means that if the user asks for analysis over 2 companies, you can invoke each agent twice, once per company.
    
    The user's question is: {objective}
    """
)

llm_planner = planner_prompt | planner_llm.with_structured_output(Plan, method="function_calling")
plan = llm_planner.invoke("Based on the past 5 years of stock market data, recommend whether to buy, hold, or sell the stocks of Apple.")

ValidationError: 4 validation errors for Plan
steps.4.action.UseAgent.agent_to_use
  Field required [type=missing, input_value={'Consolidate': {'query_t...y risks, and caveats.'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
steps.4.action.UseAgent.query_to_send
  Field required [type=missing, input_value={'Consolidate': {'query_t...y risks, and caveats.'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
steps.4.action.Respond.response
  Field required [type=missing, input_value={'Consolidate': {'query_t...y risks, and caveats.'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
steps.4.action.Consolidate.query_to_send
  Field required [type=missing, input_value={'Consolidate': {'query_t...y risks, and caveats.'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

In [ ]:
plan.steps

[Step(step_number=1, action=UseAgent(agent_to_use='fundamental_analysis_agent', query_to_send='Perform a comprehensive fundamental analysis of Apple Inc. (AAPL) over the last 5 years. Include revenue growth, earnings per share, profit margins (gross, operating, net), cash flow (operating, free), balance sheet strength (debt levels, liquidity), return metrics (ROE, ROIC), capital allocation (dividends, buybacks, acquisitions), product pipeline risks, competitive positioning, management effectiveness, and any material risks or red flags. Provide a concise narrative plus a few data points and trends.')),
 Step(step_number=2, action=UseAgent(agent_to_use='technical_analysis_agent', query_to_send='Perform a 5-year technical analysis of Apple Inc. (AAPL). Cover price trend, major moving averages (e.g., 50-day, 200-day, longer if relevant), RSI, MACD, volume patterns, volatility, support and resistance levels, and notable chart patterns. Assess current technical stance and momentum, and ident

### Replanner

In [ ]:
replanner_prompt = ChatPromptTemplate.from_template(
    """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{past_steps}

Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that. Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan."""
)

llm_replanner = replanner_prompt | planner_llm.with_structured_output(Plan)

In [ ]:
class State(TypedDict):
    query: str
    tickers: List[str]
    plan: Plan
    fundamental_analysis_agent_reply: str
    technical_analysis_agent_reply: str
    search_agent_reply: str
    steps_done: List[Step]